In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ipywidgets as widgets
from IPython.display import display, clear_output

class CanteenQueueSimulation:
    def __init__(self):
        self.lambda_input = widgets.FloatText(
            value=5.0,
            description='λ:',
            tooltip='Интенсивность потока посетителей'
        )
        
        self.p_input = widgets.FloatText(
            value=10.0,
            description='p:',
            tooltip='Параметр обслуживания'
        )
        
        self.q_input = widgets.FloatText(
            value=0.7,
            description='q:',
            tooltip='Доля посетителей, берущих два блюда',
            min=0,
            max=1
        )
        
        self.calculate_button = widgets.Button(
            description='Вычислить',
            button_style='primary'
        )
        
        self.output = widgets.Output()
        self.graph_output = widgets.Output()
        
        self.calculate_button.on_click(self.calculate)
        
        display(widgets.VBox([
            widgets.HTML('<h3>Модель работы столовой</h3>'),
            widgets.Label('Интенсивность потока посетителей (λ):'),
            self.lambda_input,
            widgets.Label('Параметр обслуживания (p):'),
            self.p_input,
            widgets.Label('Доля посетителей, берущих два блюда (q):'),
            self.q_input,
            self.calculate_button,
            self.output,
            self.graph_output
        ]))
    
    def draw_state_graph(self, lambda_rate, mu, rho):
        with self.graph_output:
            clear_output(wait=True)
            
            n_states = min(max(int(1/rho) + 3, 4), 8)
            
            G = nx.DiGraph()
            
            states = [f'S{i}\n({i} клиент{"ов" if i>1 else ""})'  if i != 0 
                     else 'S0\n(Пусто)' for i in range(n_states)]
            G.add_nodes_from(range(n_states))
            
            edges = []
            for i in range(n_states-1):
                edges.append((i, i+1, f'λ={lambda_rate:.1f}'))
                edges.append((i+1, i, f'μ={mu:.1f}'))
            
            G.add_edges_from([(u, v) for u, v, _ in edges])
            
            plt.figure(figsize=(12, 6))
            
            pos = {i: (i, 0) for i in range(n_states)}
            
            nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                                 node_size=2000, node_shape='o')
            
            edge_pos = nx.draw_networkx_edges(G, pos, edge_color='gray',
                                            arrowsize=20, arrowstyle='->',
                                            connectionstyle='arc3,rad=0.3')
            
            nx.draw_networkx_labels(G, pos, {i: states[i] for i in range(n_states)},
                                  font_size=10)
            
            edge_labels = {}
            for (u, v, label) in edges:
                edge_labels[(u, v)] = label
            
            nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels,
                                       label_pos=0.3)
            
            plt.title(f'Граф состояний системы (ρ={rho:.2f})')
            plt.axis('off')
            plt.tight_layout()
            plt.show()
    
    def calculate(self, button):
        with self.output:
            clear_output()
            
            try:
                lambda_rate = self.lambda_input.value
                p = self.p_input.value
                q = self.q_input.value
            except ValueError:
                print("Ошибка: Проверьте правильность введенных данных.")
                return

            mu = p / (1 + q)
            if lambda_rate >= mu:
                print("Система неустойчива.")
                return

            rho = lambda_rate / mu
            W_q = rho / (mu * (1 - rho))
            W = W_q + 1 / mu
            L_q = lambda_rate * W_q

            print("Система устойчива.")
            print(f"Коэффициент загрузки (ρ): {rho:.4f}")
            print(f"Средняя длина очереди (L_q): {L_q:.4f}")
            print(f"Среднее время пребывания в системе (W): {W:.4f} единиц времени")
            print(f"Среднее время ожидания в очереди (W_q): {W_q:.4f} единиц времени")
            
            self.draw_state_graph(lambda_rate, mu, rho)

simulation = CanteenQueueSimulation()